In [1]:
import pandas as pd
import numpy as np
import sklearn.decomposition as dec
import warnings
warnings.filterwarnings('ignore')

# Principal Component Analysis

## Read the data

First of all, we read the data.

In [2]:
train_data = pd.read_csv('../datasets/preprocessed/train.csv', sep=',')
test_data = pd.read_csv('../datasets/preprocessed/test.csv', sep=',')
print('Train data size ' + str(train_data.shape))
print('Test data size ' + str(test_data.shape))
data = pd.concat([train_data, test_data],ignore_index=False)
data.drop(columns=['Id'],inplace=True)

Train data size (1006, 48)
Test data size (432, 48)


In [3]:
data.head()

,MSSubClass,MSZoning,LotArea,LotShape,LandContour,LotConfig,Neighborhood,Condition1,BldgType,HouseStyle,...,Fence,MiscVal,SaleType,SaleCondition,SalePrice,MasVnr,SecondFloor,Baths,Porch,Pool
0,G,RH,0.185945,1.0,Lvl,Inside,Edwards,Artery,1Fam,2Story,...,0.0,0.0,WD,Normal,Level2,0.0,1.0,0.4,True,0.0
1,A,RL,0.198890,1.0,Lvl,Inside,NAmes,Norm,1Fam,1Story,...,1.0,0.0,WD,Family,Level2,1.0,0.0,0.0,True,0.0
2,L,RL,0.260616,1.0,Lvl,Corner,NridgHt,Norm,Twnhs,1Story,...,0.0,0.0,New,Partial,Level4,1.0,0.0,0.4,True,0.0
3,A,RL,0.251230,1.0,Lvl,Inside,NAmes,Norm,1Fam,1Story,...,1.0,0.0,WD,Abnorml,Level1,1.0,0.0,0.0,False,0.0
4,E,RL,0.174186,1.0,Lvl,Inside,SWISU,Norm,1Fam,1.5Fin,...,0.0,0.0,WD,Normal,Level2,0.0,1.0,0.4,True,0.0


## Perform PCA

We select the numerical columns, and prepare the Principal Component Analysis.

In [4]:
pca = dec.PCA()
X = data.select_dtypes(include=[np.number])
X.head()

,LotArea,LotShape,OverallQual,YearBuilt,ExterQual,TotalBsmtSF,HeatingQC,CentralAir,GrLivArea,BedroomAbvGr,...,KitchenQual,TotRmsAbvGrd,Fireplaces,GarageArea,Fence,MiscVal,MasVnr,SecondFloor,Baths,Pool
0,0.185945,1.0,0.555556,0.384058,0.333333,0.209607,0.50,0.0,0.249879,0.375,...,0.333333,0.333333,0.000000,0.215827,0.0,0.0,0.0,1.0,0.4,0.0
1,0.198890,1.0,0.444444,0.637681,0.333333,0.315034,1.00,1.0,0.193143,0.250,...,0.333333,0.333333,0.000000,0.182734,1.0,0.0,1.0,0.0,0.0,0.0
2,0.260616,1.0,0.888889,0.985507,1.000000,0.530880,1.00,1.0,0.330275,0.125,...,1.000000,0.416667,0.333333,0.756835,0.0,0.0,1.0,0.0,0.4,0.0
3,0.251230,1.0,0.444444,0.644928,0.333333,0.385215,0.50,1.0,0.217528,0.250,...,0.333333,0.333333,0.000000,0.345324,1.0,0.0,1.0,0.0,0.0,0.0
4,0.174186,1.0,0.666667,0.405797,0.333333,0.268559,0.75,1.0,0.275954,0.375,...,0.333333,0.333333,0.333333,0.155396,0.0,0.0,0.0,1.0,0.4,0.0


In [5]:
pca.fit(X)

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

We print the explained variance (individual and cumulated) and the singular values. As we want to keep around 85% of the variance, we'll keep the first 6 variables.

In [6]:
print(pca.explained_variance_ratio_)

[0.25564357 0.20193523 0.16161059 0.11836411 0.0770537  0.03889735
 0.03436969 0.02321467 0.01844822 0.01651332 0.01022641 0.00916847
 0.00774689 0.00735747 0.00548387 0.00372641 0.00318777 0.00296715
 0.00209089 0.00123523 0.00075898]


In [7]:
print(pca.explained_variance_ratio_.cumsum())

[0.25564357 0.4575788  0.61918939 0.7375535  0.8146072  0.85350456
 0.88787425 0.91108892 0.92953714 0.94605046 0.95627687 0.96544534
 0.97319223 0.9805497  0.98603357 0.98975998 0.99294775 0.9959149
 0.99800579 0.99924102 1.        ]


In [8]:
print(pca.singular_values_)

[22.17968559 19.71260381 17.63489423 15.09204065 12.17684259  8.65163089
  8.13253197  6.68373544  5.95820126  5.63709247  4.43608081  4.20035905
  3.86101551  3.76272271  3.24849001  2.67783282  2.47674573  2.38950511
  2.00587244  1.54174163  1.20851711]


We perform PCA over the dataset, X, and store it in the variable X_new.

In [9]:
X_new = pca.transform(X)

Now we print the components of each variable, to see what original columns each of the PCA variables depend on. For example, the first one depends mainly on 'KitchenQual', 'TotRmsAbvGrd' and 'Fireplaces'. The second one, on 'LotArea' and 'OverallQual', and the third one does so on 'GarageArea' and 'Fence'.

In [10]:
print(X.columns)

Index(['LotArea', 'LotShape', 'OverallQual', 'YearBuilt', 'ExterQual',
       'TotalBsmtSF', 'HeatingQC', 'CentralAir', 'GrLivArea', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageArea', 'Fence', 'MiscVal', 'MasVnr', 'SecondFloor', 'Baths',
       'Pool'],
      dtype='object')


In [11]:
np.set_printoptions(suppress=True)
pca.components_.T

array([[ 0.0611993 ,  0.0176374 ,  0.06782456,  0.02118109, -0.03863043,
         0.1838949 ,  0.22598559,  0.0365898 ,  0.38397309,  0.06983425,
        -0.17120007,  0.407443  ,  0.57357872,  0.44606464, -0.07186159,
         0.05000208,  0.11526298, -0.04179948, -0.06153459, -0.02834023,
        -0.01710126],
       [-0.4204259 , -0.03408977, -0.86449398, -0.13054026, -0.20751343,
         0.0607377 ,  0.07136187, -0.02717881,  0.01996511, -0.03580645,
        -0.01255311,  0.02171608,  0.04644228,  0.02387111, -0.00440792,
         0.00105314, -0.00213608, -0.00428958,  0.00361115,  0.00239787,
         0.00289171],
       [ 0.17868804,  0.05374833, -0.03771794, -0.03298339, -0.18880937,
         0.15125827, -0.00196549, -0.06500071, -0.06589966,  0.09002026,
        -0.18658135, -0.24401824, -0.11438053,  0.02647612, -0.52561077,
         0.67577647,  0.12638897, -0.04384778, -0.16258671, -0.044413  ,
        -0.00905928],
       [ 0.25605254, -0.04765514, -0.02374771, -0.09094915

In [12]:
data_new = pd.DataFrame(data=X_new[0:,0:6],
                        index=[i for i in range(len(X_new))],
                        columns=[i for i in range(6)])

In [13]:
data_new

,0,1,2,3,4,5
0,-0.753748,0.636941,-0.164025,-0.204247,0.549835,0.539191
1,-0.117306,-0.688384,-0.493263,0.796882,-0.008604,-0.239237
2,0.787114,-0.469485,-0.626444,-0.272131,-0.467151,0.305142
3,-0.172825,-0.708999,-0.480148,0.864432,0.145394,-0.106748
4,-0.488369,0.642514,-0.179286,-0.202405,0.023691,-0.091447
...,...,...,...,...,...,...
1433,-0.171814,-0.338120,0.821891,-0.260273,0.125090,-0.189923
1434,-1.004679,0.596731,-0.125332,0.722194,0.295212,0.767793
1435,-0.256558,-0.557312,-0.499222,-0.003437,1.051756,0.523491
1436,-0.526012,-0.374721,-0.061153,-0.341447,-0.036702,0.006344


In [14]:
train_data = data_new.loc[:1005]
test_data = data_new.loc[1006:]
train_data['Id'] = [x for x in range(len(train_data))]
test_data['Id'] = [x for x in range(len(test_data))]
print('Train data size ' + str(train_data.shape))
print('Test data size ' + str(test_data.shape))
train_data.to_csv('../datasets/preprocessed/train_pca.csv', index = None, header=True)
test_data.to_csv('../datasets/preprocessed/test_pca.csv', index = None, header=True)

Train data size (1006, 7)
Test data size (432, 7)
